In [2]:
import importlib

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import module as m
importlib.reload(m) 

<module 'module' from 'c:\\Users\\TUF\\OneDrive\\Documents\\Code\\Vs Code\\Python_DataWarehouse\\Layers\\module.py'>

In [3]:
cust = pd.read_csv("../Dataset/CRM/customers.csv.gz")
orde = pd.read_csv("../Dataset/CRM/orders.csv.gz")
menu = pd.read_csv("../Dataset/CRM/menu_items.csv.gz")
ord_itm = pd.read_csv("../Dataset/CRM/order_items.csv.gz")
res = pd.read_csv("../Dataset/CRM/restaurants.csv.gz")
deli = pd.read_csv("../Dataset/CRM/delivery_partners.csv.gz")

In [4]:
emp = pd.read_csv("../Dataset/ERP/employees.csv.gz")
inv = pd.read_csv("../Dataset/ERP/inventory.csv.gz")
kic = pd.read_csv("../Dataset/ERP/kitchen_logs.csv.gz")
sup_itm = pd.read_csv("../Dataset/ERP/supplier_items.csv.gz")
sup = pd.read_csv("../Dataset/ERP/suppliers.csv.gz")

### CRM Tables

In [5]:
kic.head()

,kitchen_log_id,order_item_id,order_id,item_id,started_at,completed_at,chef_id,status
0,KL00000001,OI0000001,O0419760,M0091,2023-12-30 09:33:52,2023-12-30 09:38:46,E00075,Completed
1,KL00000002,OI0000002,O0290838,M0374,2025-08-01 03:05:21,2025-08-01 03:09:14,E00491,Completed
2,KL00000003,OI0@00003,NaN,M0090,2025-07-03 05:11:18,2025-07-03 05:16:23,E00345,Completed
3,KL00000004,OI0000004,O0158943,M0363,2023-05-23 19:02:02,2023-05-23 19:03:09,E00498,Completed
4,nan✓,OI0000005,O0152568,M0059,NaN,2024-09-30 08:22:19,E00224,Completed


In [133]:
kitchen_log = pd.DataFrame()
# kitchen_log_id
kit_id = kic['kitchen_log_id'].astype('str').apply(lambda x: m.letter_cleaning(m.word_cleaning(m.value_cleaning(x))))
kitchen_log['kitchen_log_id'] = ['KL0'+i[2::] if len(i) == 9 else i if len(i) == 10 else None for i in kit_id]
# order_item_id
ord_itm_id = kic['order_item_id'].astype('str').apply(lambda x: m.letter_cleaning(m.word_cleaning(m.value_cleaning(x))))
ord_itm_id1 = [ 'OI0'+i[2::] if len(i) == 8 else i if len(i) == 9 else None for i in ord_itm_id]
ord_itm = pd.read_pickle("../Layers/Silver/CRM/order_item.pkl")
ord_itm_id2 = set(ord_itm['order_item_id'])
kitchen_log['order_item_id'] = [i if i in ord_itm_id2 else None for i in ord_itm_id1]
# order_id
ord_id = kic['order_id'].astype('str').apply(lambda x: m.letter_cleaning(m.word_cleaning(m.value_cleaning(x))))
ord_id1 = [ 'O0'+i[1::] if len(i) == 7 else i if len(i) == 8 else None for i in ord_id]
ordr = pd.read_pickle("../Layers/Silver/CRM/order.pkl")
ord_id2 = set(ordr['order_id'])
kitchen_log['order_id'] = [i if i in ord_id2 else None for i in ord_id1 ]
# item_id
item_id = kic['item_id'].astype('str').apply(lambda x: m.letter_cleaning(m.word_cleaning(m.value_cleaning(x))))
item_id1 = ['M0'+i[1::] if len(i) == 4 else i if len(i) == 5 else None for i in item_id]
menu_itm = pd.read_pickle("../Layers/Silver/CRM/menu_item.pkl")
item_id2 = set(menu_itm['item_id'])
kitchen_log['item_id'] = [i if i in item_id2 else None for i in item_id1 ]
# started_at
start_at = kic['started_at'].astype('str').apply(lambda x: m.word_cleaning(m.value_cleaning_sc(x)).replace('T',' '))
start_at2 = [i if len(i) == 19 else None for i in start_at]
kitchen_log['started_at'] = pd.to_datetime(start_at2, format = "%Y-%m-%d %H:%M:%S")
# completed_at
comple_at = kic['completed_at'].astype('str').apply(lambda x: m.word_cleaning(m.value_cleaning_sc(x)).replace('T',' '))
comple_at2 = [i if len(i) == 19 else None for i in comple_at]
kitchen_log['completed_at'] = pd.to_datetime(comple_at2, format = "%Y-%m-%d %H:%M:%S")

kitchen_log['started_at'] = kitchen_log['started_at'].where(kitchen_log['started_at'] < kitchen_log['completed_at'],pd.NaT)
# chef_id
chef_id = kic['chef_id'].astype('str').apply(lambda x: m.letter_cleaning(m.word_cleaning(m.value_cleaning(x))))
chef_id1 = ['E0'+i[1::] if len(i) == 5 else i if len(i) == 6 else None for i in chef_id]
emp = pd.read_pickle("../Layers/Silver/ERP/employee.pkl")
chef_id2 = set(emp['emp_id'])
kitchen_log['chef_id'] = [ i if i in chef_id2 else None for i in chef_id1]
# stauts
status = kic['status'].astype('str').apply(lambda x: m.word_cleaning(m.value_cleaning_sc(x)))
kitchen_log['status'] = [i if i in ('Completed', 'Started', 'Cancelled') else None for i in status]

In [134]:
kitchen_log

,kitchen_log_id,order_item_id,order_id,item_id,started_at,completed_at,chef_id,status
0,KL00000001,OI0000001,O0419760,M0091,2023-12-30 09:33:52,2023-12-30 09:38:46,E00075,Completed
1,KL00000002,OI0000002,O0290838,M0374,2025-08-01 03:05:21,2025-08-01 03:09:14,E00491,Completed
2,KL00000003,OI0000003,None,M0090,2025-07-03 05:11:18,2025-07-03 05:16:23,E00345,Completed
3,KL00000004,OI0000004,O0158943,M0363,2023-05-23 19:02:02,2023-05-23 19:03:09,E00498,Completed
4,None,OI0000005,O0152568,M0059,NaT,2024-09-30 08:22:19,E00224,Completed
...,...,...,...,...,...,...,...,...
1002995,KL00999996,OI0999996,O0470886,M0169,2025-07-14 07:46:42,2025-07-14 07:49:56,E00079,Completed
1002996,KL00999997,OI0999997,O0029421,M0069,2025-03-09 17:41:08,2025-03-09 17:44:32,E00367,Completed
1002997,KL00999998,OI0999998,O0033883,M0339,2023-12-07 06:30:54,2023-12-07 06:38:38,E00163,Completed
1002998,KL00999999,OI0999999,O0060674,M0191,2024-09-17 10:52:21,2024-09-17 10:55:51,E00037,Completed
